### Install required dependencies

In [7]:
!pip install -qU langchain
!pip install -qU openai
!pip install -qU cohere
!pip install -qU faiss-cpu
!pip install -qU tiktoken
!pip install -qU pypdf
!pip install -qU sentence_transformers
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.8 MB/s eta 0:00:00


### Import the required Dependencies

In [8]:
#
import os
import openai
from getpass import getpass
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

### Set up the required api keys

In [13]:
os.environ["COHERE_API_KEY"] = getpass("Cohere API Key:")
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

Cohere API Key:··········
OpenAI API Key:··········


### Upload the required documents in pdf format in the folder created

In [15]:
#os.mkdir("Documenation")

In [11]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

### Load the pdf

In [16]:
pdf_folder_path = "/content/Documenation"
loader = PyPDFDirectoryLoader(pdf_folder_path)
docs = loader.load()
#
print(len(docs))
#
print(docs[1].page_content)

22
Large Language Model
Query Classification
Reranking
DLM -based
monoT5
monoBERT
RankLLaMA
TILDE• 
• 
• 
• 
• Retrieval
Original Query
BM25
Contriever
LLM -Embedder
Query Rewriting
Query Decomposition
HyDE
Hybrid Search
HyDE+Hybrid Search• 
• 
• 
• 
• 
• 
• 
• 
•   Summarization
Extractive
Recomp
BM25
Contriever
Abstractive
LongLLMlingua
SelectiveContext
Recomp• 
• 
• 
• 
• 
• 
• 
• 
Vector Database
Milvus
Faiss
Weaviate
Qdrant
Chroma• 
• 
• 
• 
• Chunking
Chunking Size
Small2big
Sliding Windows
Chunk Metadata• 
• 
• 
• 
 
Embedding
LLM -Embedder
intfloat/e5
BAAI/bge
Jina-embeddings -v2
Gte
all-mpnet -base -v2• 
• 
• 
• 
• 
•     
Repacking
Sides
Forward
Reverse• 
• 
•  Evaluation
General Performance
Specific Domains
Retrieval Capability• 
• 
•  
Fine -tune
Disturb
Random
Normal• 
• 
•    
Retrieval SourceFigure 1: Retrieval-augmented generation workflow. This study investigates the contribution of
each component and provides insights into optimal RAG practices through extensive exper

### Split the documents into smaller chunks

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(docs)
print(len(texts))

211


### Define Embeddings Model

In [18]:
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Set up the base vector store retriever

In [19]:
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

### Retrieve the most relevant context from the vector store based on the query(No Reranking Applied)

In [20]:
query = "According to Kelly and Williams what is ethics?"
docs = retriever.get_relevant_documents(query)
pretty_print_docs(docs)

Document 1:

whether the model generations include gold answers instead of strictly exact matching [74].
Towards RAG capabilities evaluation, we adopt four metrics from RAGAs, including Faithfulness ,
Context Relevancy ,Answer Relevancy , and Answer Correctness . Faithfulness measures how
factually consistent the generated answer is with the retrieved context. An answer is considered
faithful if all claims made can be directly inferred from the provided context. Context Relevancy
----------------------------------------------------------------------------------------------------
Document 2:

GAs [ 51], including Faithfulness, Context Relevancy, Answer Relevancy, and Answer Correctness.
Additionally, we measured Retrieval Similarity by computing the cosine similarity between retrieved
documents and gold documents.
We used accuracy as the evaluation metric for the tasks of Commonsense Reasoning, Fact Checking,
and Medical QA. For Open-Domain QA and Multihop QA, we employed token-level F1

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


### The Generation part of RAG Pipeline

In [21]:
#
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k",temperature=0.1)
#
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [23]:
# %%time
print(qa.run(query=query))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Applying Reranking with CohereRerank

In [24]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
#
compressed_docs = compression_retriever.get_relevant_documents(query)
pretty_print_docs(compressed_docs)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereRerank`.
  warn_deprecated(


Document 1:

whether the model generations include gold answers instead of strictly exact matching [74].
Towards RAG capabilities evaluation, we adopt four metrics from RAGAs, including Faithfulness ,
Context Relevancy ,Answer Relevancy , and Answer Correctness . Faithfulness measures how
factually consistent the generated answer is with the retrieved context. An answer is considered
faithful if all claims made can be directly inferred from the provided context. Context Relevancy
----------------------------------------------------------------------------------------------------
Document 2:

faithful if all claims made can be directly inferred from the provided context. Context Relevancy
evaluates how relevant the retrieved context is to the original query. Answer Relevancy assesses the
21
----------------------------------------------------------------------------------------------------
Document 3:

pertinence of the generated answer to the original query. Answer Correctness involves

### Generation — RAG Pipeline using compressor retriever

In [25]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=compression_retriever )

In [ ]:
%%time
#
print(qa.run(query=query))

Credits [Advanced RAG- Cohere Re-Ranker🇰](https://medium.aiplanet.com/advanced-rag-cohere-re-ranker-99acc941601c)

